In [1]:
import sys
sys.path.insert(0, '/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6/site-packages')
sys.path

['/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6/site-packages',
 '/home/mnetlab/anaconda3/envs/pp_fl/lib/python36.zip',
 '/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6',
 '/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6/lib-dynload',
 '',
 '/home/mnetlab/.local/lib/python3.6/site-packages',
 '/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6/site-packages',
 '/home/mnetlab/.local/lib/python3.6/site-packages/IPython/extensions',
 '/home/mnetlab/.ipython']

In [2]:
# Modified fedavg for our input
from __future__ import absolute_import, division, print_function, unicode_literals

# Other import
import sys
import os
import tensorflow as tf
from data.preprocessing import preprocessing_for_training, separate_and_preprocess_for_simple_fed, evaluate_with_new_model
from data.read_data import read_data, read_setting
from data.data_utils import load_cifar10_data, train_test_label_to_categorical
from model.model import init_model, init_load_model, record_history, training_once, print_result_for_fed
from model.operation import broadcast_to_device, caculate_delta, aggregate_add, aggregate_division_return 
from config.environment import gpu_decision, zmq_bind, zmq_connect

from math import floor
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

import time
import multiprocessing as mp
import tensorflow as tf
import zmq

# for gpu memory
import xml.etree.ElementTree
import subprocess

Using TensorFlow backend.


In [ ]:
# Connect zmq socket port
socket = zmq_connect()
# Decide gpu
gpu_decision()
# Read detailed settings from json file
detailed_setting = read_setting()
# Load CIFAR-10 data
train_images, train_labels, test_images, test_labels = load_cifar10_data()
# Transfer train and test label to be categorical
train_label, test_label = train_test_label_to_categorical(train_labels, test_labels)

training_info = detailed_setting["training_info"]

# Define our model
is_master = True 
model_m, history_total = init_model(is_master)

start = time.time()

for _ in range(training_info["num_round"]):
    print("\n" + "\033[1m" + "Round: " + str(_))
    start_training = time.time()
    
    for device in range(training_info["num_device"]):
        print("\033[0m" + "Device:", device, "model_" + str(device))
        message = "{}:{}".format(_, device)
        socket.send(bytes(message, encoding = "utf8"))
        response = socket.recv()
        print("response: %s" % response)  
        
    while True:
        socket.send(b"aggregate")
        message = socket.recv()
        print("Received: %s" % message)
        if(message == b"finish"): break
            
    start_load_w = time.time()
    training_time = start_load_w - start_training
            
    is_master = False
    if(_ == 0):
        # Define and initialize an estimator model
        for device in range(training_info["num_device"]):
            globals()['model_{}'.format(device)] = tf.keras.models.load_model('parallel/model_{}.h5'.format(device))
    else:
        for device in range(training_info["num_device"]):
            globals()['model_{}'.format(device)].load_weights('parallel/model_{}.h5'.format(device))                                 
        
    loadweight_time = time.time() - start_load_w
    
    # Aggregate all delta weight on device 0 (Addition)
    for device in range(1, training_info["num_device"]):
        aggregate_add(globals()['model_{}'.format(device)], model_0)

    # Aggregate all delta weight on device 0 (Division) and return total delta weight to center device
    aggregate_division_return(model_0, model_m, training_info["num_device"])

    # Evaluate with new weight
    history_temp = evaluate_with_new_model(_, training_info, model_m, test_images, test_label)
    model_m.save('parallel/model_m.h5')

    # Record each round accuracy
    record_history(history_temp, history_total)
    
    round_time = time.time() - start_training

print("training time: ", (time.time() - start))
print_result_for_fed(history_total)



Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Round: 0
Device: 0 model_0
response: b'ok'
Device: 1 model_1
response: b'ok'
Device: 2 model_2
response: b'ok'
Device: 3 model_3
response: b'ok'
Received: b'finish'
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/mnetlab/Jeff/model/operation.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  model_0.layers[layer].get_weights()))
/home/mnetlab/Jeff/model/operation.py:52: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.divide(model_0.layers[layer].get_weights(), num_device)
/home/mnetlab/Jeff/model/operation.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=objec

10000/10000 [==============================] - 1s 148us/sample - loss: 2.6534 - acc: 0.1000

Round 0 
Accuracy: 0.1, Loss: 2.6534363338470457

Round: 1
Device: 0 model_0
response: b'ok'
Device: 1 model_1
response: b'ok'
Device: 2 model_2
response: b'ok'
Device: 3 model_3
response: b'ok'
Received: b'finish'
10000/10000 [==============================] - 1s 54us/sample - loss: 2.2718 - acc: 0.1166

Round 1 
Accuracy: 0.1166, Loss: 2.2717701152801513

Round: 2
Device: 0 model_0
response: b'ok'
Device: 1 model_1
response: b'ok'
Device: 2 model_2
response: b'ok'
Device: 3 model_3
response: b'ok'
Received: b'finish'
10000/10000 [==============================] - 1s 55us/sample - loss: 1.0672 - acc: 0.6896

Round 2 
Accuracy: 0.6896, Loss: 1.0671874478340149

Round: 3
Device: 0 model_0
response: b'ok'
Device: 1 model_1
response: b'ok'
Device: 2 model_2
response: b'ok'
Device: 3 model_3
response: b'ok'
Received: b'finish'
10000/10000 [==============================] - 1s 65us/sample - loss: 0.